In [1]:
import tensorflow as tf
import numpy as np
from os import path, getcwd, chdir
import pandas as pd
import matplotlib.pyplot as plt
import csv
import tarfile


In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler

def DATA(file1,file2,W):
  dd=100
  
  file=file1
  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]

  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  uu=np.zeros((len(Y1)-dd,1))
  uu[:,0]=Y1[dd:]

  
  X_Data=np.zeros((len(Y1)-W,W))
  for i in range(len(Y1)-W):
    for j in range(i,i+W):
      X_Data[i,j-i]=Y1[j]

  file=file2
  
  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]

  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  vv=np.zeros((len(Y1)-dd,1))
  vv[:,0]=Y1[dd:]

  
  Y_Data=np.zeros((len(Y1)-W,W))
  for i in range(len(Y1)-W):
    for j in range(i,i+W):
      Y_Data[i,j-i]=Y1[j]

  xx = np.reshape(uu, [len(uu),])
  yy = np.reshape(vv, [len(vv),])

  
  return(X_Data,Y_Data)

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler

def DATA2(file1,file2,file3,W):
  dd=100

  file=file1
  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]

  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  uu=np.zeros((len(Y1)-dd,1))
  uu[:,0]=Y1[dd:]


  X_Data=np.zeros((len(Y1)-W,W))
  for i in range(len(Y1)-W):
    for j in range(i,i+W):
      X_Data[i,j-i]=Y1[j]

  file=file2
 
  f=open(file,"r")
  lines=f.readlines()
  Y1x=[]
  X1=[]
  for x in lines:
    X1.append(float(x.split()[0]))
    Y1x.append(float(x.split()[1]))


  file=file3
  f=open(file,"r")
  lines=f.readlines()
  Y2x=[]
  X1=[]
  for x in lines:
    X1.append(float(x.split()[0]))
    Y2x.append(float(x.split()[1]))

  Ynew_Data=np.zeros((len(Y1)-W,W,2))
  for i in range(len(Y1)-W):
      for j in range(i,i+W):

        Ynew_Data[i,(j-i),0]=Y1x[j]
        Ynew_Data[i,(j-i),1]=Y2x[j]

  return(X_Data,Ynew_Data)

In [ ]:
def G8_relation():
  df = pd.read_csv (r'G8FBCinfo.csv')
  print (df)
  Diameter=np.zeros((len(df)+1,1))
  Q=np.zeros((len(df)+1,1))

  Di=df['Diii']
  volumeflow=df['vol_florate Ca=1']
  for i in range(len(df)):
    Diameter[i+1,0]=Di[i]
    Q[i+1,0]=volumeflow[i]

  BIF = pd.read_csv (r'bif_relation8.csv')
  BIF1=BIF['mother']
  BIF2=BIF['daughter1']
  BIF3=BIF['daughter2'] 

  mother=np.zeros((len(BIF1)+1,1))
  daughter1=np.zeros((len(BIF1)+1,1))
  daughter2=np.zeros((len(BIF1)+1,1))
  for i in range(len(BIF1)):
    mother[i+1,0]=BIF1[i]
    daughter1[i+1,0]=BIF2[i]
    daughter2[i+1,0]=BIF3[i]
  return(mother,daughter1,daughter2,BIF1,BIF2,BIF3,Q)

In [ ]:
#generating model for prediction of daughter vessel quantity from mother vessel quantity 
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

mother,daughter1,daughter2,BIF1,BIF2,BIF3,Q=G8_relation()
model_store=[]

# for each mother vessel you need to have time window 
# time window can be obtained from method described in section 3.3 
window=[[2,50],
[4,40],
[5,100],
[6,40],
[7,80],
[8,80],
[9,100],
[10,30],
[11,90],
[12,80],
[13,100],
[14,50],
[15,110],
[16,100],
[17,70],
[18,50],
[19,100],
[20,60],
[21,120]]

#matching_bif=16

# bi identify the bifurcation we are working on 
for bi in range(1,22):
  if bi !=3 :
    for ss in range(len(window)):
      if bi == window[ss][0]:
        W=window[ss][1]
    ST=[]
    SK=[]
    KU=[]
    for mm in range(matching_bif,matching_bif+1):
      print('bif=',bi,'W=',W)
      for i in range(len(BIF1)+1):
        if i==bi:
          mom=mother[i,0]
          dau1=daughter1[i,0]
          dau2=daughter2[i,0]
          d1=0
          d2=0
          m=mom
          if Q[int(dau1),0]/Q[int(mom),0]>Q[int(dau2),0]/Q[int(mom),0]:
            d1=dau1
            d2=dau2
          elif Q[int(dau1),0]/Q[int(mom),0]<Q[int(dau2),0]/Q[int(mom),0]:
            d1=dau2
            d2=dau1
        
          file1='fort.'+str(6001+int(10*m))
          file2='fort.'+str(6001+int(10*d1))
          file3='fort.'+str(6001+int(10*d2))
          print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0],'biger_Q',d1,file1,file2)

      X_Data,Y_Data=DATA2(file1,file2,file3,W)

      X_Data=np.reshape(X_Data,(X_Data.shape[0],X_Data.shape[1],1))
      Y_Data=np.reshape(Y_Data,(Y_Data.shape[0],Y_Data.shape[1],2))

      model = Sequential()

      model.add(LSTM(units=100, return_sequences=True, input_shape=(W,1)))
      model.add(Dropout(0.1))

      model.add(LSTM(units=100, return_sequences=True))
      model.add(Dropout(0.1))

      model.add(LSTM(units=100, return_sequences=True))
      model.add(Dropout(0.1))

      model.add(Dense(units = W))

      model.compile(optimizer='adam',
                            loss='mse',
                            metrics=['mse'])
      model.summary()
      history = model.fit(
                    X_Data, Y_Data, epochs=3000            
          )

      model_store.append([W,model])
  

In [ ]:
number_bif =21
for i in range(number_bif):
  model_store[i].save('TwoBif_Bif'+str(i+1)+'/my_model') 
model.save('HtTwobif'+str(4)+'/my_model') 

In [ ]:
#untar the saved models
!tar -xvf TwoBif_Bif1.tar.gz
!tar -xvf TwoBif_Bif2.tar.gz
!tar -xvf TwoBif_Bif3.tar.gz
!tar -xvf TwoBif_Bif4.tar.gz
!tar -xvf TwoBif_Bif5.tar.gz
!tar -xvf TwoBif_Bif6.tar.gz
!tar -xvf TwoBif_Bif7.tar.gz
!tar -xvf TwoBif_Bif8.tar.gz
!tar -xvf TwoBif_Bif9.tar.gz
!tar -xvf TwoBif_Bif10.tar.gz
!tar -xvf TwoBif_Bif11.tar.gz
!tar -xvf TwoBif_Bif12.tar.gz
!tar -xvf TwoBif_Bif13.tar.gz
!tar -xvf TwoBif_Bif14.tar.gz
!tar -xvf TwoBif_Bif15.tar.gz
!tar -xvf TwoBif_Bif16.tar.gz
!tar -xvf TwoBif_Bif17.tar.gz
!tar -xvf TwoBif_Bif18.tar.gz
!tar -xvf TwoBif_Bif19.tar.gz
!tar -xvf TwoBif_Bif20.tar.gz
!tar -xvf TwoBif_Bif21.tar.gz

In [ ]:
#info1 shows time window for each bifurcation 
info1=[[2,50],
[4,40],
[5,100],
[6,40],
[7,80],
[8,80],
[9,100],
[10,30],
[11,90],
[12,80],
[13,100],
[14,50],
[15,110],
[16,100],
[17,70],
[18,50],
[19,100],
[20,60],
[21,120]]
modelss=[]
for j in range(len(info1)):
    print(j)
    modelss.append([info1[j][0],tf.keras.models.load_model('content/Twobif'+str(info1[j][0])+'/my_model')])

In [ ]:
# extend the quantity of interest over time based on what we discuss in the section 3.3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import random

import csv
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
import sys

#------->define mother vessel 
bi = 10  #give the bifurcation that you want to work on them  
for i in range(len(BIF1)+1):
  if i==bi:
    
    mom=mother[i,0]
    dau1=daughter1[i,0]
    dau2=daughter2[i,0]
    d1=0
    d2=0
    m=mom
    if Q[int(dau1),0]/Q[int(mom),0]>Q[int(dau2),0]/Q[int(mom),0]:
      d1=dau1
      d2=dau2
    elif Q[int(dau1),0]/Q[int(mom),0]<Q[int(dau2),0]/Q[int(mom),0]:
      d1=dau2
      d2=dau1
    file1='fort.'+str(5001+int(10*m))
    file2='fort.'+str(5001+int(10*d1))
    file3='fort.'+str(5001+int(10*d2))

print('file1',file1)


dd=100

EPO=500
window=0
Sk=[]
Kt=[]
Me=[]
St=[]
Res=[]
file = file1
for io in range (1):
  
  window = 450 # obtained time window from code1-....
  print('window = ',window)
  #####
  
  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]
  
  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  apple_training_processed=np.zeros((len(Y1)-dd,1))
  apple_training_processed[:,0]=Y1[dd:]

  scaler = MinMaxScaler(feature_range = (0, 1))

  apple_training_scaled = scaler.fit_transform(apple_training_processed)

  features_set = []
  labels = []
  for i in range(0, len(apple_training_scaled)-1-2*window):
      features_set.append(apple_training_scaled[i:i+window, 0])
      labels.append(apple_training_scaled[i+window:i+2*window, 0])


  features_set, labels = np.array(features_set), np.array(labels)


  features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
  labels = np.reshape(labels, (labels.shape[0], labels.shape[1], 1))


  model = Sequential()

  model.add(LSTM(units=100, return_sequences=True, input_shape=(features_set.shape[1], 1)))
  model.add(Dropout(0.1))


  model.add(LSTM(units=100, return_sequences=True))
  model.add(Dropout(0.1))


  model.add(LSTM(units=100, return_sequences=True))
  model.add(Dropout(0.1))


  model.add(Dense(units = 1))
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  model.summary()
  history=model.fit(features_set, labels, epochs = EPO, batch_size = 32,callbacks=[EarlyStoppingByLossVal()])



  from sklearn.preprocessing import MinMaxScaler


  #file = 'fort.5061'
  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]
  
  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  data_actual=np.zeros((len(Y1)-dd,1))
  data_actual[:,0]=Y1[dd:]

  #######
  plt.plot(data_actual[:], color='blue', label='Actual Apple Stock Price')


  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  Y1=[]
  Z1=[]
  
  for x in lines:
      X1.append(float(x.split()[0]))
      Y1.append(float(x.split()[1]))
  data=np.zeros((len(Y1)-dd,1))
  data[:,0]=Y1[dd:]

  #######
  print('SGH',np.shape(data))

  only_pred=[]
  for iter in range(6):
    print('iter',iter)
    scaler = MinMaxScaler(feature_range = (0, 1))

    data_scaled = scaler.fit_transform(data)
    data_set = []
    for i in range(0, len(data)-2*window-1):
        data_set.append(data_scaled[i:i+window, 0])

    data_set = np.array(data_set)
    data_set=np.reshape(data_set,(data_set.shape[0], data_set.shape[1], 1))


    pred_scaled=model.predict(data_set[len(data_set)-1,:,:].reshape(1,window,1))
    #print('jjjj',pred_scaled[0])
    data=np.array(data)
    data=data.tolist()
    
    for i in range(len(pred_scaled[0])):
      ff=pred_scaled[0,i][0]
      ff=ff.reshape(-1, 1)
      pred = scaler.inverse_transform(ff)
  
      data.append([pred[0,0]])
      only_pred.append([pred[0,0]])

      

    data=np.array(data)

  # You can sketch your predicted curve in time with original curve and also compare the statisticals  
  plt.plot(data[:] , color='red', label='Predicted Apple Stock Price')
  plt.plot(data_actual[:], color='blue', label='Actual Apple Stock Price')

  Sk.append((window,skew(data[len(uu):])))
  Kt.append((window,kurtosis(data[len(uu):])))
  Me.append((window,np.mean(data[len(uu):])))
  St.append((window,np.std(data[len(uu):])))
  print('window',window,'skewness=',skew(data),'kurtosis=',kurtosis(data))
  print('window',window,'True skewness=',skew(data_actual),'Truekurtosis=',kurtosis(data_actual))
  print('window',window,'mean data',np.mean(data),'std data',np.std(data))
  print('window',window,'True mean data',np.mean(data_actual),'True std',np.std(data_actual))

In [ ]:
# Predicted the quantity of interest in each daughter vessel given that in mother vessel 
print('first step')
for sa in range(len(info1)):
  if bi == info1[sa][0]:
    W=info1[sa][1]

dd1=data
ff=dd1.tolist()
print('bi=',bi,'W=',W)
lo=[]
for i in range(len(ff)):
  lo.append(ff[i][0])

f=[]
for j in range(1,int(len(lo)/W)):
  f.append(np.array(lo[(j-1)*W:W*j]))

a1,a2=np.shape(f)
f2=np.zeros((a1,a2,1))

f2[:,:,0]=f[:][:]

nn=len(f2)

for eb in range(len(modelss)):
  if bi==modelss[eb][0]:
    mm=modelss[eb][1]


k1=[]
k2=[]
for j in range(0,nn):
  pred=mm.predict(f2[j,:,:].reshape(1,W,1))
  pred=pred.reshape(W,2)
  for i in range(W):
    k1.append(pred[i,0])
    k2.append(pred[i,1])


In [ ]:
# plot your predicted curve in each daughter vessel and compare with the DSR of those daughter vessels 
plt.plot(k1),plt.plot(k2)
plt.plot(data)

f=open(file2,"r")
lines=f.readlines()
X1=[]
Y1=[]
Z1=[]
for x in lines:
    X1.append(float(x.split()[0]))
    Y1.append(float(x.split()[1]))
plt.plot(Y1)
f=open(file3,"r")
lines=f.readlines()
X11=[]
Y11=[]
Z11=[]

for x in lines:
    X11.append(float(x.split()[0]))
    Y11.append(float(x.split()[1]))
plt.plot(Y11)

print('mean d1=',np.mean(k1),'True mean d1',np.mean(Y1),'mean d2=',np.mean(k2),'True Skew d2',np.mean(Y11))
print('std d1=',np.std(k1),'True std d1',np.std(Y1),'std d2=',np.std(k2),'True std d2',np.std(Y11))
print('skewness d1=',skew(k1),'True Skew d1',skew(Y1),'skewness d2=',skew(k2),'True Skew d2',skew(Y11))
print('kurtosis d1=',kurtosis(k1),'True kurtosis d1',kurtosis(Y1),'kurtosis d2=',kurtosis(k2),'True kurtosis d2',kurtosis(Y11))
print('***************************************************************************')
print('Statistics----Mean')
print(np.mean(k1),'  ',np.mean(Y1),)
print(np.mean(k2),'  ',np.mean(Y11))

print('Statistics----std')
print(np.std(k1),'     ',np.std(Y1))
print(np.std(k2),'     ',np.std(Y11))

print('Statistics----skew')
print(skew(k1),'  ',skew(Y1))
print(skew(k2),'  ',skew(Y11))

print('Statistics----kurtosis')
print(kurtosis(k1),'   ',kurtosis(Y1))
print(kurtosis(k2),'   ',kurtosis(Y11))